In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint 
from sklearn.tree import export_graphviz 
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydot

In [2]:
tempIemisions = pd.read_csv("DataSetFinal.csv", index_col=0)
dummiesCountry=pd.get_dummies(tempIemisions['Country'],drop_first=1)
tempIemisions=pd.concat([tempIemisions,dummiesCountry],axis=1)
tempIemisions=tempIemisions.drop(['Country'], axis=1)

In [3]:
#fem X totes les columnes menys temp i y nomes temp el axis=1 es per indicar
#que es una label de les columnes
X = tempIemisions.drop('AverageTemperature', axis=1)
y = tempIemisions['AverageTemperature']

#dividim el dataSet en test 20% i entrenament 80%
#el random_state es la llavor que s'utilitzarà per randomitzar 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
#creem l'algorisme del arbre de decisions i l'entrenem
regressor = DecisionTreeRegressor()
regressor.fit(X_train,y_train)


#fem les prediccions
y_pred = regressor.predict(X_test)
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
comparacio=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
print (comparacio)

Mean Absolute Error: 1.6633936955063717
Mean Squared Error: 5.034015990610329
Root Mean Squared Error: 2.243661291418633
      Actual  Predicted
6549  12.831      9.647
1737   6.847      4.497
595   21.939     22.608
793    2.101      6.176
3612   4.824      5.721
2786   8.855      8.470
4726  18.706     18.031
3535  15.300     15.401
3567  11.100      9.765
6184  13.177     15.777
2328   6.820      7.974
7051  17.054     14.711
6891  11.121     15.336
2362  11.783      9.202
955   23.640     22.185
2746   4.611      5.220
4805  24.936     23.675
7433  12.705     14.049
88    12.270     12.106
4077   6.785      8.267
5660  20.295     20.261
4332  -7.296     -0.583
1160  14.623     12.682
5403   7.381      8.598
497   16.508     17.605
761   21.941     21.642
2126  -9.870     -4.163
179   -2.043     -0.032
4257   7.049      6.514
128   12.455     16.030
...      ...        ...
6746  10.986     12.581
2898  26.179     26.076
2261  18.445     17.571
4229  16.217     15.143
1998  13.972   

In [4]:
parameters = {'max_depth':range(10, 200, 10), 
              'min_samples_split':range(2,70,5),
              'min_samples_leaf':range(2,70,5)}
scoring = 'neg_mean_absolute_error'
regressor1 = DecisionTreeRegressor()
gscv=GridSearchCV(regressor1,parameters,scoring,verbose=1)
gscv.fit(X_train,y_train)

#resultats
print('Millor combinacio:', gscv.best_params_)
y_pred = gscv.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Fitting 3 folds for each of 3724 candidates, totalling 11172 fits
Millor combinacio: {'max_depth': 60, 'min_samples_leaf': 2, 'min_samples_split': 32}
Mean Absolute Error: 1.3709749378117153
Mean Squared Error: 3.3979482665410523
Root Mean Squared Error: 1.8433524531518795


[Parallel(n_jobs=1)]: Done 11172 out of 11172 | elapsed:  3.0min finished


In [5]:
parameters = {'max_depth':sp_randint(2, 500), 
              'min_samples_split':sp_randint(2, 150),
              'min_samples_leaf':sp_randint(2, 150)}
scoring = 'neg_mean_absolute_error'
n_iter=2000
regressor2 = DecisionTreeRegressor()
rscv=RandomizedSearchCV(regressor2,parameters,n_iter,scoring,verbose=1)
rscv.fit(X_train,y_train)
print('Millor combinacio:', rscv.best_params_)
y_pred = rscv.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Fitting 3 folds for each of 2000 candidates, totalling 6000 fits
Millor combinacio: {'max_depth': 475, 'min_samples_leaf': 4, 'min_samples_split': 23}
Mean Absolute Error: 1.3635898068667394
Mean Squared Error: 3.420122176302294
Root Mean Squared Error: 1.8493572332846604


[Parallel(n_jobs=1)]: Done 6000 out of 6000 | elapsed:  1.3min finished


In [7]:
#decision tree per imprimir

X = tempIemisions.drop('AverageTemperature', axis=1)
y = tempIemisions['AverageTemperature']

#dividim el dataSet en test 20% i entrenament 80%
#el random_state es la llavor que s'utilitzarà per randomitzar 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
#creem l'algorisme del arbre de decisions i l'entrenem
regressor = DecisionTreeRegressor(max_depth=4)
regressor.fit(X_train,y_train)

#imprimir arbre
dot_data = StringIO()

export_graphviz(regressor, out_file='regressor.dot',  
                filled=True, rounded=True,
                special_characters=True)
(graph,) = pydot.graph_from_dot_file('regressor.dot')
graph.write_png('regressor.png')